In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

x=np.arange(-1, 1, step = 0.02)
plt.ylim(-1, 1)
arctan=np.arctan(x*2)*2/np.pi
plt.plot(x,arctan)
plt.plot(x,x)

plt.show()
